In [ ]:
# import os
# # Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# # For example:
# # spark_version = 'spark-3.4.0'
# spark_version = 'spark-3.4.0'
# os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# # Start a SparkSession
# import findspark
# findspark.init()


0% [Working]
            
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease

0% [Connecting to archive.ubuntu.com] [Connecting to security.ubuntu.com (185.1
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease

0% [Connecting to archive.ubuntu.com (185.125.190.36)] [Waiting for headers] [W
                                                                               
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.lau

In [1]:
!pip install findspark
!pip install pyspark
!pip install pyarrow
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c428890ff2c1271e6dc2f9933ced0649c2aeecd8f3a9a536a457c4047846f22f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:00


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('homes')


In [16]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql(
    """
    SELECT CAST(avg(price) as DECIMAL(16,2))
    FROM homes
    WHERE bedrooms == 4
    ;


    """
).show()


+---------------------------------+
|CAST(avg(price) AS DECIMAL(16,2))|
+---------------------------------+
|                        299661.01|
+---------------------------------+



In [19]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# group by date_built, where bedrooms == 3 and bathrooms == 3

spark.sql(
    """
    SELECT CAST(avg(price) as DECIMAL(16,2)), date_built
    FROM homes
    WHERE bedrooms == 3 AND bathrooms == 3
    GROUP BY date_built
    """
).show()


+---------------------------------+----------+
|CAST(avg(price) AS DECIMAL(16,2))|date_built|
+---------------------------------+----------+
|                        288770.30|      2015|
|                        295962.27|      2013|
|                        290852.27|      2014|
|                        293683.19|      2012|
|                        290555.07|      2016|
|                        292859.62|      2010|
|                        291117.47|      2011|
|                        292676.79|      2017|
+---------------------------------+----------+



In [20]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
# group by date_built GTE 2000

spark.sql(
    """
    SELECT CAST(avg(price) as DECIMAL(16,2)), date_built
    FROM homes
    WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
    GROUP BY date_built
    """
).show()


+---------------------------------+----------+
|CAST(avg(price) AS DECIMAL(16,2))|date_built|
+---------------------------------+----------+
|                        297609.97|      2015|
|                        303676.79|      2013|
|                        298264.72|      2014|
|                        307539.97|      2012|
|                        293965.10|      2016|
|                        285010.22|      2010|
|                        276553.81|      2011|
|                        280317.58|      2017|
+---------------------------------+----------+



In [28]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql(
    """
    SELECT CAST(avg(price) as DECIMAL(16,2)), view
    FROM homes
    GROUP BY view
    HAVING avg(price) >= 350000
    ORDER BY view desc

    """
).show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------------------------------+----+
|CAST(avg(price) AS DECIMAL(16,2))|view|
+---------------------------------+----+
|                       1026669.50| 100|
|                       1061201.42|  99|
|                       1053739.33|  98|
|                       1129040.15|  97|
|                       1017815.92|  96|
|                       1054325.60|  95|
|                       1033536.20|  94|
|                       1026006.06|  93|
|                        970402.55|  92|
|                       1137372.73|  91|
|                       1062654.16|  90|
|                       1107839.15|  89|
|                       1031719.35|  88|
|                       1072285.20|  87|
|                       1070444.25|  86|
|                       1056336.74|  85|
|                       1117233.13|  84|
|                       1033965.93|  83|
|                       1063498.00|  82|
|                       1053472.79|  81|
+---------------------------------+----+
only showing top

In [29]:
# 7. Cache the the temporary table homes.
spark.sql("""cache table homes""")

DataFrame[]

In [30]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [31]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql(
    """
    SELECT CAST(avg(price) as DECIMAL(16,2)), view
    FROM homes
    GROUP BY view
    HAVING avg(price) >= 350000
    ORDER BY view desc

    """
).show()


print("--- %s seconds ---" % (time.time() - start_time))


+---------------------------------+----+
|CAST(avg(price) AS DECIMAL(16,2))|view|
+---------------------------------+----+
|                       1026669.50| 100|
|                       1061201.42|  99|
|                       1053739.33|  98|
|                       1129040.15|  97|
|                       1017815.92|  96|
|                       1054325.60|  95|
|                       1033536.20|  94|
|                       1026006.06|  93|
|                        970402.55|  92|
|                       1137372.73|  91|
|                       1062654.16|  90|
|                       1107839.15|  89|
|                       1031719.35|  88|
|                       1072285.20|  87|
|                       1070444.25|  86|
|                       1056336.74|  85|
|                       1117233.13|  84|
|                       1033965.93|  83|
|                       1063498.00|  82|
|                       1053472.79|  81|
+---------------------------------+----+
only showing top

In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [33]:
# 11. Read the parquet formatted data.
p_home_sales_df = spark.read.parquet("home_sales_partitioned")

In [34]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_homes_p')

In [35]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql(
    """
    SELECT CAST(avg(price) as DECIMAL(16,2)), view
    FROM p_homes_p
    GROUP BY view
    HAVING avg(price) >= 350000
    ORDER BY view desc

    """
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------------------------------+----+
|CAST(avg(price) AS DECIMAL(16,2))|view|
+---------------------------------+----+
|                       1026669.50| 100|
|                       1061201.42|  99|
|                       1053739.33|  98|
|                       1129040.15|  97|
|                       1017815.92|  96|
|                       1054325.60|  95|
|                       1033536.20|  94|
|                       1026006.06|  93|
|                        970402.55|  92|
|                       1137372.73|  91|
|                       1062654.16|  90|
|                       1107839.15|  89|
|                       1031719.35|  88|
|                       1072285.20|  87|
|                       1070444.25|  86|
|                       1056336.74|  85|
|                       1117233.13|  84|
|                       1033965.93|  83|
|                       1063498.00|  82|
|                       1053472.79|  81|
+---------------------------------+----+
only showing top

In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql(""" uncache table homes """)

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('homes'):
  print('table still cached')
else:
  print('all clear')


all clear
